# Web-scrapping project

### We would like to extract gas price from a website. We initially attempted to run the "request" command, but it did not work. Then, we used "header" to make the host website understand, it's not requested by a robot...it is now able to extract the data

### Import necessary libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Request the content for the gas price

In [2]:
url = 'https://finance.yahoo.com/quote/NG%3DF/futures?p=NG%3DF'
hh = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}
r = requests.get(url, headers=hh)
print(r.status_code)

200


When the code is 200, it means it works well and data can be extracted

### Get the table from the website

In [4]:
soup = BeautifulSoup(r.content, 'lxml')
total = soup.find('table', class_='W(100%) M(0) BdB Bdc($seperatorColor)')

### Get the columns for date and price

In [8]:
result = pd.DataFrame(columns=['A', 'B'])
trs = total.find_all('tr')[1:]
row = 1
for tr in trs:
    tds = tr.find_all('td')
    result.loc[row] = [tds[1].text, tds[2].text]
    row = row + 1
    
result.tail(20)

,A,B
80,"Henry Hub Natural Gas Futures,M",2.949 8:00PM EDT
81,"Henry Hub Natural Gas Futures,J",2.979 8:00PM EDT
82,"Henry Hub Natural Gas Futures,J",3.019 8:00PM EDT
83,"Henry Hub Natural Gas Futures,A",3.059 8:00PM EDT
84,"Henry Hub Natural Gas Futures,N",3.198 8:00PM EDT
85,"Henry Hub Natural Gas Futures,S",3.074 8:00PM EDT
86,Natural Gas Oct 29,4.26 9:06AM EDT
87,"Henry Hub Natural Gas Futures,D",3.35 8:00PM EDT
88,"Henry Hub Natural Gas Futures,J",3.48 8:00PM EDT
89,"Henry Hub Natural Gas Futures,F",3.45 8:00PM EDT


### Data cleaning: show the rows that we do NOT want

In [10]:
result2 = result.loc[result['A'].str.contains('Henry Hub')]
result2[:]

,A,B
72,"Henry Hub Natural Gas Futures,N",3.128 8:00PM EDT
75,"Henry Hub Natural Gas Futures,D",3.278 8:00PM EDT
76,"Henry Hub Natural Gas Futures,J",3.403 8:00PM EDT
77,"Henry Hub Natural Gas Futures,F",3.376 8:00PM EDT
78,"Henry Hub Natural Gas Futures,M",3.311 8:00PM EDT
79,"Henry Hub Natural Gas Futures,A",2.971 8:00PM EDT
80,"Henry Hub Natural Gas Futures,M",2.949 8:00PM EDT
81,"Henry Hub Natural Gas Futures,J",2.979 8:00PM EDT
82,"Henry Hub Natural Gas Futures,J",3.019 8:00PM EDT
83,"Henry Hub Natural Gas Futures,A",3.059 8:00PM EDT


### Data cleaning: show the rows that we want to keep

In [14]:
result3 = result.loc[~ result['A'].str.contains('Henry Hub')].reset_index()
result3.drop('index', axis=1, inplace=True)

# To help python to understand the date, we put 01 at the beginning of each date entry:
result3['A'] = [w.strip().replace('Natural Gas', '01') for w in result3['A']]

# Conver the date entry to pandas datetime:
result3['A'] = pd.to_datetime(result3['A'])
result3['B'] = [w.split()[0] for w in result3['B']]
result3

,A,B
0,2022-11-01,7.785
1,2022-10-01,7.727
2,2022-12-01,7.939
3,2023-01-01,8.069
4,2023-02-01,7.753
5,2023-03-01,6.714
6,2023-04-01,5.327
7,2023-05-01,5.201
8,2023-06-01,5.25
9,2023-07-01,5.307
